In [58]:
import numpy as np
from IPython.display import clear_output


class Sudoku:
    
    def __init__(self,board):
        self.size=9
        self.board=board#np.zeros((self.size,self.size))

    def checkSquare(self,row,col,n):
        col*=3
        row*=3
        for yy in range(3):
            for xx in range(3):
                if self.board[row+yy,col+xx]==n:
                    return True
        return False
    
    def checkColumn(self,col,n):
        for yy in range(self.size):
            if self.board[yy,col]==n:
                return True
        return False
    
    def checkRow(self,row,n):
        for xx in range(self.size):
            if self.board[row,xx]==n:
                return True
        return False
        
    def findPossibleSolutions(self):
        moves=np.zeros((self.size,self.size,self.size))
        for y in range(self.size):
            for x in range(self.size):
                if self.board[y,x]==0:
                    for n in range(self.size):
                        exists=self.checkSquare(int(y/3),int(x/3),n+1)
                        if not exists:
                            exists=self.checkColumn(x,n+1)
                        if not exists:
                            exists=self.checkRow(y,n+1)
                        if not exists:
                            moves[y,x,n]=1
        self.moves=moves
    
    def simplify(self):
        pass
    
    def checkEnd(self):
        for y in range(self.size):
            for x in range(self.size):
                if self.board[y,x]==0:
                    return False
        return True
    
    def tryFill(self):
        find=False
        #cerca soluzioni uniche
        for y in range(self.size):
            for x in range(self.size):
                if self.board[y,x]==0:
                    nn=0
                    moves=0
                    for n in range(self.size):
                        if self.moves[y,x,n]==1:
                            nn=n+1
                            moves+=1
                    if moves==1:
                        self.board[y,x]=nn
                        find=True
        #cerca unica soluzione possibile per quadrato
        for y in range(int(self.size/3)):
            for x in range(int(self.size/3)):
                for n in range(self.size):
                    exists=self.checkSquare(y,x,n+1)
                    if not exists:
                        possibility=0
                        cell=(-1,-1)
                        for yy in range(3):
                            for xx in range(3):
                                if self.board[y*3+yy,x*3+xx]==0 and self.moves[y*3+yy,x*3+xx,n]==1:
                                    cell=(y*3+yy,x*3+xx)
                                    possibility+=1                                    
                        if possibility==1:
                            self.board[cell[0],cell[1]]=n+1
                            find=True    

        #cerca unica soluzione possibile per riga
        for y in range(self.size):
            for n in range(self.size):
                exists=self.checkRow(y,n+1)
                if not exists:
                    possibility=0
                    cell=(-1,-1)
                    for x in range(self.size):
                        if self.board[y,x]==0 and self.moves[y,x,n]==1:
                            cell=(y,x)
                            possibility+=1
                    if possibility==1:
                        self.board[cell[0],cell[1]]=n+1
                        find=True    

        #cerca unica soluzione possibile per colonna
        for x in range(self.size):
            for n in range(self.size):
                exists=self.checkColumn(x,n+1)
                if not exists:
                    possibility=0
                    cell=(-1,-1)
                    for y in range(self.size):
                        if self.board[y,x]==0 and self.moves[y,x,n]==1:
                            cell=(y,x)
                            possibility+=1
                    if possibility==1:
                        self.board[cell[0],cell[1]]=n+1
                        find=True    
        
                        
        return find
    
    def printBoard(self):
        output="|-------|-------|-------|\n"
        for y in range(self.size):            
            for x in range(self.size):
                if (x%3)==0:
                    output+="| "
                if self.board[y,x]>0:
                    output+=str(self.board[y,x])+" "
                else:
                    output+=". "
            output+="|\n"
            if y>1 and((y+1)%3==0 or y==8):
                output+="|-------|-------|-------|\n"

        print(output)
        print("\n")

    def printMoves(self):
        output=""
        for y in range(self.size):
            for x in range(self.size):
                for n in range(self.size):
                    output+=str(int(self.moves[y,x,n]))
                output+=" "
            output+='\n'
        print(output)
        print("---------\n\n")


#facile
board=np.array([[9,1,7,2,5,4,0,0,0],
                [4,0,2,0,8,0,0,0,0],
                [6,5,0,0,0,3,4,0,0],
                [0,0,3,0,9,0,2,5,6],
                [5,0,0,7,0,0,3,0,9],
                [2,0,0,0,0,5,0,7,1],
                [0,2,0,5,3,0,7,6,0],
                [3,7,0,1,6,0,0,9,8],
                [0,0,0,0,0,0,0,3,0]])

#difficile
board=np.array([[0,0,0,0,5,1,0,0,0],
                [5,6,1,9,0,0,0,0,0],
                [4,0,0,7,0,0,0,0,0],
                [0,0,2,0,0,5,4,0,0],
                [0,4,5,0,0,0,0,0,8],
                [1,9,0,0,4,0,0,0,3],
                [0,8,0,0,2,7,0,3,1],
                [6,0,0,0,0,0,0,2,0],
                [0,5,0,8,0,0,6,4,9]])


import time 
import timeit


s=Sudoku(board)
s.printBoard()
fine=False
steps=0
start = timeit.default_timer()
while not fine:
    #clear_output(wait=True)
    steps+=1
    print(steps)
    s.findPossibleSolutions()
    #s.printMoves()
    
    f=s.tryFill()
    s.printBoard()
    #time.sleep(2)
    fine=s.checkEnd()
    if (not fine) and (not f):
        break

stop = timeit.default_timer()
print(steps)
#s.printBoard()
#s.printMoves()
if fine:
    print("resolved")
else:
    print("no solution at the moment")
        
#Your statements here


print('Time: ', stop - start)
        

        

|-------|-------|-------|
| . . . | . 5 1 | . . . |
| 5 6 1 | 9 . . | . . . |
| 4 . . | 7 . . | . . . |
|-------|-------|-------|
| . . 2 | . . 5 | 4 . . |
| . 4 5 | . . . | . . 8 |
| 1 9 . | . 4 . | . . 3 |
|-------|-------|-------|
| . 8 . | . 2 7 | . 3 1 |
| 6 . . | . . . | . 2 . |
| . 5 . | 8 . . | 6 4 9 |
|-------|-------|-------|



1
|-------|-------|-------|
| . . . | . 5 1 | . . . |
| 5 6 1 | 9 . . | . . . |
| 4 . . | 7 . . | . . . |
|-------|-------|-------|
| 8 . 2 | . . 5 | 4 . . |
| . 4 5 | . . . | . . 8 |
| 1 9 6 | . 4 8 | . . 3 |
|-------|-------|-------|
| 9 8 4 | 6 2 7 | 5 3 1 |
| 6 1 . | 5 . . | 8 2 . |
| 2 5 7 | 8 1 3 | 6 4 9 |
|-------|-------|-------|



2
|-------|-------|-------|
| . . . | 4 5 1 | . . . |
| 5 6 1 | 9 . . | . . . |
| 4 . . | 7 . . | . . 5 |
|-------|-------|-------|
| 8 . 2 | . . 5 | 4 . . |
| . 4 5 | . . 9 | . . 8 |
| 1 9 6 | 2 4 8 | 7 5 3 |
|-------|-------|-------|
| 9 8 4 | 6 2 7 | 5 3 1 |
| 6 1 3 | 5 9 4 | 8 2 7 |
| 2 5 7 | 8 1 3 | 6 4 9 |
|-